In [2]:
import time
import numpy as np
import pandas as pd

from scipy import sparse
from scipy.stats.mstats import gmean
from datetime import datetime
from sklearn import preprocessing
from scipy.stats import skew, boxcox,boxcox_normmax
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from bayes_opt import BayesianOptimization
from sklearn.metrics import log_loss

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

seed = 1234


# Load Data

In [4]:
data_path = "../input/"
# train_X = pd.read_csv(data_path + 'train_BM_0331.csv')
test_X = pd.read_csv(data_path + 'test_BM_0331.csv')

# train_y = np.ravel(pd.read_csv('../input/' + 'labels_BrandenMurray.csv'))
# train_y = to_categorical(train_y)

# ntrain = train_X.shape[0]
sub_id = test_X.listing_id.astype('int32').values

In [15]:
data_path = "../"
# train_X = pd.read_csv(data_path + 'train_BM_MB_add03052240_desc.csv')
test_X = pd.read_csv(data_path + 'test_2ndkares.csv',header=None)
# train_y = np.ravel(pd.read_csv(data_path + 'labels_BrandenMurray.csv'))

# all_features = features_to_use + desc_sparse_cols + feat_sparse_cols
print test_X.shape

(74659, 3)


In [16]:
tmp = test_X/33

In [17]:
# now = datetime.now()
sub_name = '../output/sub_2ndKeras_100bagging_1_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

out_df = pd.DataFrame(tmp)
out_df.columns = ["low", "medium", "high"]
out_df["listing_id"] = sub_id
out_df.to_csv(sub_name, index=False)

In [18]:
tmp

,low,medium,high,listing_id
0,0.184460,0.668446,0.147093,7142618
1,0.992886,0.006776,0.000339,7210040
2,0.982917,0.016380,0.000703,7174566
3,0.135420,0.573688,0.290892,7191391
4,0.801136,0.193431,0.005433,7171695
5,0.810894,0.185443,0.003662,7225206
6,0.997962,0.001891,0.000148,7200075
7,0.247026,0.498261,0.254714,7145074
8,0.957108,0.041677,0.001215,7193645
9,0.992094,0.007647,0.000259,7147703
